In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2

In [2]:
import glob
antman = glob.glob('CAX_Superhero_Train/Ant-Man/*')
aquaman = glob.glob('CAX_Superhero_Train/Aquaman/*')
avengers = glob.glob('CAX_Superhero_Train/Avengers/*')
batman = glob.glob('CAX_Superhero_Train/Batman/*')
blackpanther = glob.glob('CAX_Superhero_Train/Black Panther/*')
captainamerica = glob.glob('CAX_Superhero_Train/Captain America/*')
catwoman = glob.glob('CAX_Superhero_Train/Catwoman/*')
ghostrider = glob.glob('CAX_Superhero_Train/Ghost Rider/*')
hulk = glob.glob('CAX_Superhero_Train/Hulk/*')
ironman = glob.glob('CAX_Superhero_Train/Iron Man/*')
spiderman = glob.glob('CAX_Superhero_Train/Spiderman/*')
superman = glob.glob('CAX_Superhero_Train/Superman/*')

In [3]:
def proc_images(antman):

    x = [] # images as arrays
    y = [] # labels 
    WIDTH = 256
    HEIGHT = 256

    for fname in antman:

        # Read and resize image
        full_size_image = cv2.imread(fname)
        x.append(cv2.resize(full_size_image, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))

        # Labels
        label = fname.split("cax_",1)[1].split("_train",1)[0]
        y.append(label)
        
    x = np.array(x)
    y = np.array(y)
    return x,y

In [4]:
x1,y1 = proc_images(antman)
x2,y2 = proc_images(aquaman)
x3,y3 = proc_images(avengers)
x4,y4 = proc_images(batman)
x5,y5 = proc_images(blackpanther)
x6,y6 = proc_images(captainamerica)
x7,y7 = proc_images(catwoman)
x8,y8 = proc_images(ghostrider)
x9,y9 = proc_images(hulk)
x10,y10 = proc_images(ironman)
x11,y11 = proc_images(spiderman)
x12,y12 = proc_images(superman)

In [5]:
x = np.vstack([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12])

In [6]:
y = np.concatenate([y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12])

In [7]:
s = pd.Series(y)
y = np.array(pd.get_dummies(s))

In [8]:
## normalize images

x = x/255.

In [11]:
from sklearn import preprocessing
from sklearn import model_selection

def model_feed(x,y):
    x_train, x_validate, y_train, y_validate = model_selection.train_test_split(x, y, test_size=0.2)
    
    return (x_train,y_train,x_validate,y_validate)

In [12]:
x_train,y_train,x_validate,y_validate = model_feed(x,y)

## CNN

In [15]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model

Using TensorFlow backend.


In [20]:
def Basic_CNN(input_shape):

    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    
    X = Conv2D(32, (7, 7), strides=(1, 1), name='conv0')(X)
    X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    
    X = Flatten()(X)
    X = Dense(12, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs=X_input, outputs=X)
    
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = Basic_CNN(x_train.shape[1:])

model.fit(x_train, y_train, validation_data=(x_validate, y_validate), epochs=10, batch_size= 50)

Train on 4346 samples, validate on 1087 samples
Epoch 1/10
4346/4346 [==============================] - 1320s - loss: 7.6090 - acc: 0.0881 - val_loss: 7.6158 - val_acc: 0.0883